# Nuovi positivi e medie mobili

*In questo notebook si rappresentano con uno scatter plot i nuovi positivi e se ne interpolano i valori con medie mobili su 7 o 14 giorni. L'asse temporale si può esprimere in mesi oppure nel numero di giorni trascorsi dall'inizio (24/02/2020) della pandemia.*

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import datetime

La funzione partiziona `xDati` in un array di array. Ciascun elemento ha lunghezza `intervalloGiorni` e il loro numero è pari a `n-intervalloGiorni+1`.

In [ ]:
def partiziona(xdati, intervalloGiorni):
    numElem = len(xdati)
    elementi = []
    for i in range(intervalloGiorni, numElem + 1):
        elementi = np.append(elementi, xdati[i-intervalloGiorni:i])
    return np.reshape(elementi, (numElem - intervalloGiorni + 1, intervalloGiorni))
    #################################################################################################################

Informazioni da inserire in input per iniziare l'elaborazione dei dati nazionali o regionali memorizzati nei file corrispondenti alla data scelta. Viene inoltre richiesta l'unità temporale da porre in ascissa.

In [ ]:
print()
print("Si intendono elaborare i nuovi positivi nazionali (i) o quelli di una regione (r)? ")
scelta = input(" :> ")
print("Inserire la data nel formato (YYYYMMDD) ")
dataISO = input(" :> ")
print("Si vuole l'asse temporale espresso in mesi (m) o in giorni trascorsi (g) dall'inizio pandemia? ")
unitaAssex = input(" :> ")

Vengono composti i nomi dei file e quindi si carica nel'array `nuoviPositivi` i dati rilevati

In [ ]:
if scelta == "i":
    regioneScelta = "ITALIA"
    nomeFile = 'datiNazionali' + dataISO
    # lettura del file csv: la colonna 8 corrisponde al campo nuovi_positivi
    nuoviPositivi = np.genfromtxt(nomeFile + '.csv', delimiter=',', skip_header=1, usecols=8, dtype=int)
else:
    print("Inserire la regione ")
    regioneScelta = input(" :> ")
    regioneScelta = regioneScelta.lower().capitalize()
    nomeFile = 'datiRegionali' + dataISO
    # lettura del file json e quindi si riporta l'array ad un array di Numpy
    with open(nomeFile + '.json') as f:
        datiGrezzi = json.load(f)
    nuoviPositivi = []
    for record in datiGrezzi:
        if record['denominazione_regione'] == regioneScelta:
            nuoviPositivi.append(record['nuovi_positivi'])
    nuoviPositivi = np.array(nuoviPositivi)

#################################################################################################################

Si definiscono le ampiezze degli intervalli entro cui calcolare le medie mobili dopo aver partizionato coerentemente l'array `nuoviPositivi`.

In [ ]:
intervalloGiorni = 14
medie = np.mean(partiziona(nuoviPositivi, intervalloGiorni), axis=1)
intervalloGiorni7 = 7
medie7 = np.mean(partiziona(nuoviPositivi, intervalloGiorni7), axis=1)


Elementi utili per la parte grafica

In [ ]:
numGiorni = len(nuoviPositivi)
codificaItaliana = dataISO[6:] + '-' + dataISO[4:6] + '-' + dataISO[:4]
massimoPositivi = np.max(nuoviPositivi)
#################################################################################################################

Inizia la parte grafica comune e quindi quella dipendente dalla scelta iniziale dell'asse temporale.

In [ ]:
# parte grafica comune
plt.rcParams['figure.figsize'] = [12, 6]
fig, ax = plt.subplots()
ax.set_title(regioneScelta + ": media mobile su " + str(intervalloGiorni7) + " e " + str(intervalloGiorni) + " giorni dei nuovi positivi")
ax.set_ylabel('nuovi positivi giornalieri')
ax.grid(which='both', color='.85', linestyle='-', linewidth=1)
# parte variabile
if unitaAssex == "m":
    # sono necessari due insiemi di dati temporali in quanto il numero dei valori medi è diverso dal numero dei dati iniziali
    date1 = [datetime.date(2020, 2, 24) + datetime.timedelta(days=i) for i in range(numGiorni-1)]
    ax.scatter(date1, nuoviPositivi[1:], s=4, color='cornflowerblue', label='nuovi positivi', zorder=3)
    date2 = [datetime.date(2020, 2, 24) + datetime.timedelta(days=int(intervalloGiorni/2)) + datetime.timedelta(days=i)
             for i in range(numGiorni-intervalloGiorni+1)]
    ax.plot(date2, medie, color='r', label="media mobile su " + str(intervalloGiorni) + " giorni", zorder=5)
    date7 = [datetime.date(2020, 2, 24) + datetime.timedelta(days=int(intervalloGiorni7/2)) + datetime.timedelta(days=i)
             for i in range(numGiorni-intervalloGiorni7+1)]
    ax.plot(date7, medie7, color='yellowgreen', linewidth=3, label="media mobile su " + str(intervalloGiorni7) + " giorni", zorder=4)
    ax.set_xlabel('date dal 24/02/2020')
    ax.text(datetime.date(2020, 6, 15), massimoPositivi/2, 'aggiornato il\n' + codificaItaliana)
else:
    days = np.linspace(1, numGiorni, numGiorni)
    ax.scatter(days, nuoviPositivi, marker='.', color='cornflowerblue', label='nuovi positivi')
    ax.plot(days[(intervalloGiorni7-1):]-(intervalloGiorni7/2), medie7, color='yellowgreen',
            linewidth=3, label="media mobile su " + str(intervalloGiorni7) + " giorni")
    ax.plot(days[(intervalloGiorni-1):]-(intervalloGiorni/2), medie, color='r', label="media mobile su " + str(intervalloGiorni) + " giorni")
    ax.set_xlabel('giorni dal 24 febbraio 2020')
    x_major_ticks = np.arange(0, numGiorni+20, 20)
    ax.set_xticks(x_major_ticks)
    ax.text(70, massimoPositivi/2, 'aggiornato il\n' + codificaItaliana)

plt.legend()
plt.show()
#################################################################################################################